# Сбор данных

In [45]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import requests
import lxml.html
from pprint import pprint

### Обработка html-страниц
### hh.ru

In [34]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.0; rv:14.0) Gecko/20100101 Firefox/14.0.1',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'ru-ru,ru;q=0.8,en-us;q=0.5,en;q=0.3',
    'Accept-Encoding':'gzip, deflate',
    'Connection':'keep-alive',
    'DNT':'1'
}

vacancies = ['python-разработчик', 'devops']
df = pd.DataFrame(columns=['vacancy', 'desription'])
for vacancy in vacancies:
    url = 'https://hh.ru/search/vacancy?clusters=true&text=' + vacancy + '&enable_snippets=true&page=0&search_field=name&no_magic=true'
    request = requests.get(url, headers=headers)
    soup = BeautifulSoup(request.text, 'lxml')
    css_selector_vacancy_urls = '.HH-LinkModifier'
    vacancy_urls = list(map(lambda x: x['href'], soup.select(css_selector_vacancy_urls)))
    for url in vacancy_urls[:2]:
        request = requests.get(url, headers=headers)
        soup = BeautifulSoup(request.text, 'lxml')
        css_selector_desription = '.g-user-content'
        vacancy_text = soup.select(css_selector_desription)[0].text
        df.loc[len(df)] = [vacancy, vacancy_text]
df

,vacancy,desription
0,python-разработчик,В настоящее время компания Itransition активно...
1,python-разработчик,SaM Solutions is looking for an experienced Py...
2,devops,Обязанности: Администрирование и поддержка L...
3,devops,"IT company specializing IT operations, operati..."


### imdb.com

In [35]:
films = ['Balto', 'Hellboy', 'Fargo', 'Aliens']
df = pd.DataFrame(columns=['title', 'overview'])
for film in films:
    url = 'https://www.imdb.com/find?ref_=nv_sr_fn&q=' + film + '&s=all'
    request = requests.get(url, headers=headers)
    soup = BeautifulSoup(request.text, 'lxml')
    css_selector_film_url = '.result_text a'
    film_url = 'https://www.imdb.com' + soup.select(css_selector_film_url)[0]['href']
    request = requests.get(film_url, headers=headers)
    soup = BeautifulSoup(request.text, 'lxml')
    css_selector_overview = '.summary_text'
    overview_text = soup.select(css_selector_overview)[0].text.strip()
    df.loc[len(df)] = [film, overview_text]
df

,title,overview
0,Balto,An outcast half-wolf risks his life to prevent...
1,Hellboy,"Based on the graphic novels by Mike Mignola, H..."
2,Fargo,"Various chronicles of deception, intrigue and ..."
3,Aliens,Ellen Ripley is rescued by a deep salvage team...


## Обработка json
### api.hh.ru
https://github.com/hhru/api

In [54]:
url = 'https://api.hh.ru/vacancies?specialization=7&per_page=1&page=0'
request = requests.get(url)
pprint(request.json())

{'alternate_url': 'https://hh.ru/search/vacancy?enable_snippets=true&items_on_page=1&specialization=7',
 'arguments': None,
 'clusters': None,
 'found': 16549,
 'items': [{'address': None,
            'alternate_url': 'https://hh.ru/vacancy/30188755',
            'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=30188755',
            'archived': False,
            'area': {'id': '1002',
                     'name': 'Минск',
                     'url': 'https://api.hh.ru/areas/1002'},
            'contacts': {'email': 'contact@shinomontaj.by',
                         'name': 'Vadim Nikolaevich',
                         'phones': [{'city': '29',
                                     'comment': 'приглашаем '
                                                'автоэлектрика-диагноста, '
                                                'автослесаря, адрес СТО: ул. '
                                                'К.Либкнехта,77А',
                                    

In [55]:
vacancy_id = request.json()['items'][0]['alternate_url'].replace('https://hh.ru/vacancy/','')
vacancy_url = 'https://api.hh.ru/vacancies/' + vacancy_id
request = requests.get(vacancy_url)
pprint(request.json())

{'accept_handicapped': False,
 'accept_incomplete_resumes': True,
 'accept_kids': False,
 'address': None,
 'allow_messages': True,
 'alternate_url': 'https://hh.ru/vacancy/30188755',
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=30188755',
 'archived': False,
 'area': {'id': '1002', 'name': 'Минск', 'url': 'https://api.hh.ru/areas/1002'},
 'billing_type': {'id': 'free', 'name': 'Бесплатная'},
 'branded_description': None,
 'code': 'Автомобильный бизнес',
 'contacts': {'email': 'contact@shinomontaj.by',
              'name': 'Vadim Nikolaevich',
              'phones': [{'city': '29',
                          'comment': 'приглашаем автоэлектрика-диагноста, '
                                     'автослесаря, адрес СТО: ул. '
                                     'К.Либкнехта,77А',
                          'country': '375',
                          'number': '33-999-11'}]},
 'created_at': '2019-02-24T13:18:43+0300',
 'department': None,
 'description': '<